# Analyse des décès en France (toutes causes confondues)
Ce notebook a pour but de mettre les chiffres de décès de 2020 en France en perspective afin d'en tirer des conclusions éclairées.

In [ ]:
# Bibliothèque d'analyse et de traitement de données
import pandas as pd
from pandas.api.types import CategoricalDtype
print("Pandas version :", pd.__version__)
import numpy as np
from datetime import datetime
# Bibliothèques de gestion des graphiques
import matplotlib.pyplot as plt
# Bibliothèques de gestion des fichiers
import glob
import os.path
# Bibliothèque de gestion du format csv pour écrire les fichiers cache
import csv
# "Constantes" ou variables globales
csv_delimiter = ';'
print ('Imports des bibliothèques terminés !')

# Données sources
Les données sources sont issues du __[Fichier des personnes décédées](https://www.data.gouv.fr/fr/datasets/fichier-des-personnes-decedees/)__ fournies en données ouvertes sur [data.gouv.fr](https://www.data.gouv.fr/fr/).

Les fichiers `deces-*.txt` doivent être téléchargés dans le dossier de travail du notebook.
Pour un traitement rapide, commencer par un petit nombre de fichiers, ex: 2002 et 2003.

**note** : Certains fichiers portent parfois quelques "caractères non imprimables" qui font échouer sa lecture et son interprétation. Dans ce cas, il faut l'éditer pour rechercher et retirer ces caractères. Je ne sais pas si c'est un problème du fichier d'origine ou si c'est dû à un parasitage du téléchargement, je n'ai pas investigué.

## Analyse et extraction
Pour accélérer le travail, nous alons lire les fichiers `deces-*.txt` qui ont un format "en largeur fixe" et en extraire les informations essentielles suivantes :
 - date de décès
 - date de naissance
 
Ces informations seront écrites dans un fichier cache au format CSV qui permettra plusieurs choses :
 1. Accélérer une éventuelle relecture des données sources : au lieu de lire pour chaque année un fichier de plus de 100 Mo, on ne lit que quelques Mo.
 1. Manipuler ces données dans un tableur indépendamment de cet exercice à des fins de recoupement, par exemple.
 1. Porter directement l'âge du décès par soustraction plutôt que la date de naissance pour éviter de refaire le calcul à chaque utilisation de la donnée.
 
Pour que le code soit compréhensible, je laisse un certain nombre de variables intermédiaires.

In [ ]:
# Lister tous les fichiers qui commencent par "deces" et sont au format texte (finissent par ".txt")
sourcelist = [f for f in glob.glob("deces*.txt")]
# Pour chaque fichier de cette liste...
for source in sourcelist:
    # on va créer un pendant au même nom, mais se terminant par .csv
    cachefile = source[0:-4] + '.csv'
    # Par contre, on ne crée ce fichier que s'il n'existe pas ou s'il est plus vieux que le fichier de données
    if not(os.path.exists(cachefile)) or os.path.getmtime(source) > os.path.getmtime(cachefile):
        if os.path.exists(cachefile):
            os.remove(cachefile)
        with open(cachefile, 'w', newline='') as cache:
            cache_writer = csv.writer(cache, delimiter = csv_delimiter, quotechar='"', quoting=csv.QUOTE_MINIMAL)
            print ('Génération du cache CSV pour le fichier : ' + source)
            # On crée une première ligne portant le nom des 2 colonnes
            cache_writer.writerow(['date', 'age'])
            # On ouvre ensuite le fichier source pour en lire toutes ses lignes en gérant les caractères accentués
            with open(source,'r', errors='replace') as f:
                lines = f.readlines()
            # Pour chaque ligne du fichier source...
            for l in lines:
                # ...on interprète la date de décès aux caractères 154 à 161 inclus (donc 162 exclus)
                txt_date_d = l[154:162]
                try:
                    date_d = datetime.strptime(txt_date_d, "%Y%m%d")
                except ValueError:
                    # Si la date n'est pas interprétable, alors on met zero
                    date_d = 0
                    continue
                # ...on interprète la date de naissance aux caractères 81 à 88
                try:
                    date_n = datetime.strptime(l[81:89], "%Y%m%d")
                except ValueError:
                    # Si la date n'est pas interprétable, alors on met zero
                    date_n = 0
                    continue
                # Ici, on peut choisir de filtrer.
                # Par exemple, on peut choisir de ne conserver que les années 20xx et 199x
                # Mais surtout, on ignore les dates non reconnues
                if date_d != 0 and date_n != 0 and (txt_date_d[0:2] == '20' or txt_date_d[0:3] == '199'):
                    # Puis on calcule l'âge qu'on écrit avec la date de décès dans le fichier csv
                    age = date_d.year - date_n.year - ((date_d.month, date_d.day) < (date_n.month, date_n.day))
                    cache_writer.writerow([txt_date_d[0:4]+'/'+txt_date_d[4:6]+'/'+txt_date_d[6:8],age])
print ('(Ré)génération des caches CSV terminée !')

## Relecture du cache
Ci-après, on lit tous les fichiers cache pour les concaténer en mémoire afin de faire notre analyse.

Pour réinitialiser toutes les données de la "dataframe" df, c'est cette cellule qu'il faut réexécuter.

In [ ]:
print ('Lecture des fichiers cache...')
# Créer une liste de tous les csv
chunks = []
cachelist = [f for f in glob.glob("deces*.csv")]
for cache in cachelist:
    # print ('Lecture de :', cache)
    chunks.append(pd.read_csv(cache,
                              sep=';',
                              encoding='latin_1',        # Spécifie l'encodage du fichier à lire
                              dtype={'age':np.int8},     # Stocke l'age dans un format court pour la performance
                              index_col=['date'],        # Crée un indexe sur la colonne date
                              parse_dates=True))         # Interprète automatiquement ce qui ressemble à une date
# Concatène le tout
df = pd.concat(chunks, axis=0, ignore_index=False)
# Affiche les premières lignes pour vérifier que tout s'est bien passé
df.tail()
# Exemples d'utilisation :
#  * Sélection de tous les décès à moins de 50 ans : df[df.age < 50]
#  * Sélection de tous les décès du mois de février 2020 : df.loc['2020-02']

## Classification
Pour se simplifier l'interprétation graphique, on va classer chaque décès dans une catégorie d'âge.

J'ai découpé en tranches plus ou moins fines...

In [ ]:
df['classe'] = pd.cut(df.age, bins=[0, 18, 25, 45, 55, 65, 75, 85, 150],
                      labels=['mineur', '18-25 ans', '26-45 ans', '46-55 ans', '56-65 ans', '66-75 ans', '76-85 ans', 'plus de 86 ans'])
df.tail()

## Nombre de décès par date par catégorie
A partir de la dataframe contenant les catégories, on crée une matrice pour compter le nombre de décès par date par catégorie. 

In [ ]:
ages_df = df.groupby(['date', 'classe']).age.count().astype(np.int16).unstack()
ages_df.tail()

# Analyses graphiques
## Analyse graphique par classe d'âge durant le confinement en France
Afin de rendre la courbe plus lisible (la débruiter), on utilise une fonction de lissage ewm moyenné avec un facteur alpha assez faible.
Analysons la période du confinement élargie de mars à mai.
Constate-t-on que pour la population au dessous de 65 ans, ne nombre de décès n'a pas varié de façon notable ?

In [ ]:
# Pour supprimer le lissage, mettre alpha=1
# _= est une petite astuce de Romain WaaY pour ne pas afficher de message parasite avec le graphique
_=ages_df.ewm(alpha=0.2).mean().loc['2020-03':'2020-05'].plot(figsize=(18,10), legend='reverse')

## Analyse graphique sur les 20 dernières années
Mettons maintenant en perspective la surmortalité du printemps 2020 par rapport aux 20 dernières années, toutes classes d'âge confondues.

D'une manière générale, on constate des pics de décès de plus en plus hauts tous les 2 à 4 ans.

Le pic de 2020 est élevé, mais semble rester dans la tendance haussière générale...

In [ ]:
_=df.loc['2000-09':'2020-09-15'].classe.groupby('date').count().plot(figsize=(18,8))
# Calcul des moyennes annuelles
an_debut = 2001
an_fin = 2021
moyennes = pd.Series([int(ages_df.loc[str(an)].mean().sum()) for an in range(an_debut,an_fin)],
                     index=[datetime.strptime(str(an), "%Y") for an in range(an_debut,an_fin)])
moyennes.plot()

In [ ]:
#Todo: superposer les courbes
#plt.figure(figsize=(18,8))
#df.loc['2009-09'].classe.groupby('date').count().plot()
#df.loc['2010-09'].classe.groupby('date').count().plot()

In [ ]:
ddf = pd.DataFrame(df.classe.groupby('date').count())
ddf.tail()

In [ ]:
#Todo: superposer les courbes
plt.figure(figsize=(18,8))
df.loc['2019-09-01':'2019-10-26'].classe.groupby('date').count().ewm(alpha=0.2).mean().plot()
df.loc['2018-09-01':'2018-10-26'].classe.groupby('date').count().ewm(alpha=0.2).mean().plot()

In [ ]:
ddf['annee'] = ddf.index.year
ddf['doy'] = ddf.index.dayofyear
ddf.tail()

In [ ]:
ddf.reset_index(inplace=True)
ddf.set_index(['annee', 'doy'], inplace=True)
ddf

In [ ]:
plt.figure(figsize=(18,18))
ddf.loc[2017].classe.plot(x=ddf.date)
ddf.loc[2018].classe.plot(x=ddf.date)
ddf.loc[2019].classe.plot(x=ddf.date)
ddf.loc[2020].classe.plot(x=ddf.date)